In [9]:
import selenium
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
import urllib.request
from urllib.error import ContentTooShortError
from tqdm import tqdm
import pandas as pd
import json
import os
from bs4 import BeautifulSoup
import time
import unittest
from Data_Collection_ASOS_v6 import Scraper

class ScraperTest(unittest.TestCase):
    def setUp(self):
        url = 'https://www.asos.com/women/sale/cat/?cid=7046&nlid=ww|sale|shop+sale+by+product|sale+view+all'
        self.driver = Chrome('./chromedriver')        
        self.driver.get(url)
        self.delay = 10
      
        

# Unit Test 1: Test whether the load_and_accept_cookies_button_works
    def test_cookies(self):
      Scraper.load_and_accept_cookies(self)
      if self.assertTrue(Scraper.unit_Test_cookies == 'True'):
        print('The load_and_accept_cookies function is working properly')

  # Unit Test 2: Test whether the load_more_products fn works  
    def test_load_pages(self):
      Scraper.load_more_products(self)
      if self.asserTrue(Scraper.unit_Test_Load == 'True'):
        print(' The load_more_products function is workng properly')
      else:
        print('The load more button was not clicked')

      #Test 2a: Did it load the right number of pages
      actual_value2 = Scraper.load_pages
      expected_value2 = Scraper.n_pages
      if self.assertEqual(expected_value2, actual_value2):
        print(f' The load_more_products will load {Scraper.n_pages}')
      else:
        print(" There were no additional pages loaded.")

    # Unit Test 3: Test whether the get_links fn works  
    def test_links(self):
        Scraper.load_and_accept_cookies()
        Scraper.get_product_links()
        actual_value = len(Scraper.shop_link_list)
        expected_value = 72
        if self.assertEqual(expected_value, actual_value):
           print(f"There are {len(self.shop_link_list)} in this list")
        else:
          print('There has been an error gathering test links')
      

        #Test 3a: Are the right links being collected
        expected_values = ['https://www.asos.com/london-rebel/london-rebel-chunky-jelly-sandals-in-black/prd/201295338?clr=black-matt&colourWayId=201295339&cid=7046',
        'https://www.asos.com/maybelline/maybelline-sky-high-lash-sensational-mascara/prd/22963786?clr=01-ultra-black&colourWayId=60440102&cid=7046',
        'https://www.asos.com/pepe-jeans/pepe-jeans-seamfree-set-in-black/grp/23758?clr=black&cid=7046']
        actual_values = Scraper.shop_link_list[0:3]
        if self.assertListEqual(expected_values, actual_values):
          print('The proper links have been collected from the website')
        else:
          print('There has been an error gathering the product links')
          pass

    
    # Unit Test 4: Test whether the get_product_data fn works  
    def test_collected_data(self):
        self.test_group_list = [{"111428846": {"name":["London Rebel chunky jelly sandals in black"],
            "image": ["https://images.asos-media.com/products/london-rebel-chunky-jelly-sandals-in-black/201295338-1-blackmatt?$n_240w$&wid=40&fit=constrain"],
            "previous_price": [" \u00a330.00"],"sale price": [" \u00a311.25"],
            "sale_percentage": ["(-62%)"],"color": ["Black matt"],
            "description": ["\nSandals by London Rebel\nThe scroll is over\nAdjustable strap\nPin-buckle fastening\nOpen toe\nFlat sole\nTexture grip tread"],
            "sizes": [ "\nUK 3\nUK 4\nUK 5\nUK 6\nUK 7\nUK 8"]}},
    {"1912185": {"name": ["Maybelline Sky High Lash Sensational Mascara"],
            "image": ["https://images.asos-media.com/products/maybelline-sky-high-lash-sensational-mascara/22963786-1-01ultrablack?$n_240w$&wid=40&fit=constrain"],
            "previous_price": [" \u00a311.50"],"sale price": [" \u00a39.00"],"sale_percentage": ["(-21%)"],
            "color": ["01 ultra black"],"description": ["\nMascara by Maybelline\nDial up your look \nDesigned to lengthen and add volume \nSpiked wand\nInfused with bamboo extract and fibres for long, full lashes\nProduct is non-returnable for hygiene reasons"],
            "sizes": [""]}}]
        
        actual_products = len(Scraper.full_product_data)
    
        # Unit Test 4a&b: Test whether the right products and the correct number of products have been collected
        if self.assertListEqual(Scraper.full_product_data, self.test_group_list) and self.assertCountEqual(72, actual_products):
          print("Scraper is properly collecting data")
        else:
          print('There has been an error with gathering product data')
          pass
    
     # Unit Test 5: Test to make sure the right data has been saved
    def test_saved_data(self):
      self.test_org_list = json.loads('ASOS_data/ASOS_Women_Org_data')
      self.test_dict = json.loads('ASOS_data/ASOS_Women_data')
      if self.assertDictEqual(Scraper.full_item_list, self.test_dict) and self.assertListEqual(Scraper.full_product_data, self.test_org_list):
        print('Scraper is properly saving data for later use')
      else:
        print('There has been an error with saving the data')
        pass
    

    # Unit Test 5: Test to make sure the right images has been saved
    def test_saved_images(self):
      self.imagetesting = ['https://images.asos-media.com/products/london-rebel-chunky-jelly-sandals-in-black/201295338-1-blackmatt?$n_240w$&wid=40&fit=constrain', 
      'https://images.asos-media.com/products/maybelline-sky-high-lash-sensational-mascara/22963786-1-01ultrablack?$n_240w$&wid=40&fit=constrain', 
      'https://images.asos-media.com/groups/pepe-jeans-seamfree-set-in-black/23758-group-1?$n_240w$&wid=40&fit=constrain',
      'https://images.asos-media.com/products/under-armour-training-heat-gear-base-layer-booty-shorts-in-grey/21887549-1-grey?$n_240w$&wid=40&fit=constrain', 
      'https://images.asos-media.com/products/stradivarius-organic-cotton-slim-mom-jean-with-stretch-in-washed-blue/200549201-1-blue?$n_240w$&wid=40&fit=constrain']
      if self.assertListEqual(Scraper.imagesrc_list, self.imagetesting ):
        print('The proper image sources were gathered')
      else:
        print(' There was a problem with the image sources')
    
      for i in range(5):
        id = Scraper.full_item_list['product_id'][i]
        if os.path.exists(f"ASOS_data/images/ASOS_image_{id}.jpg"):
            return True
        else:
          print('There was an error retrieving {id} picture')

    def tearDown(self):
      self.driver.quit()
  
    def complete_testing(self):
      self.setUp()
      self.test_cookies()
      self.test_load_pages()
      self.tearDown()
      self.setUp()
      self.test_links()
      self.test_collected_data()
      self.test_saved_data()
      self.test_saved_images()
      self.tearDown()

if __name__ == "__main__":
  ScraperTest = ScraperTest()

ScraperTest.complete_testing()